# Tarea 6

https://medium.com/@suci/running-pyspark-on-jupyter-notebook-with-docker-602b18ac4494

## Ejercicio 1

In [1]:
import pandas as pd

In [2]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc = SparkContext()
sqlContext = SQLContext(sc)

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local[3]").getOrCreate()

In [3]:
# from pyspark.sql import SQLContext
# sqlContext = SQLContext(sc)

In [6]:
employees = spark.read.csv('/home/jovyan/northwind/employees.csv', header =True)
orders = spark.read.csv('/home/jovyan/northwind/orders.csv', header =True)

In [7]:
type(employees)

pyspark.sql.dataframe.DataFrame

In [8]:
employees.select("firstname").show()

+---------+
|firstname|
+---------+
|    Nancy|
|   Andrew|
|    Janet|
| Margaret|
|   Steven|
|  Michael|
|   Robert|
|    Laura|
|     Anne|
+---------+



In [9]:
employees.describe()

DataFrame[summary: string, employeeid: string, lastname: string, firstname: string, title: string, titleofcourtesy: string, birthdate: string, hiredate: string, address: string, city: string, region: string, postalcode: string, country: string, homephone: string, extension: string, photo: string, notes: string, reportsto: string, photopath: string]

# Ejercicio 1a
1a. ¿Cuántos “jefes” hay en la tabla empleados? ¿Cuáles son estos jefes: número de empleado, nombre, apellido, título, fecha de nacimiento, fecha en que iniciaron en la empresa, ciudad y país? (atributo reportsto, ocupa explode en tu respuesta)

In [10]:
employees.createOrReplaceTempView("employees") 

In [11]:
res = spark.sql("""SELECT employeeid,firstname,lastname,titleofcourtesy,
birthdate,hiredate,city,country FROM employees where employeeid 
IN ( select distinct reportsto from employees where reportsto >0)""")
res.toPandas()

,employeeid,firstname,lastname,titleofcourtesy,birthdate,hiredate,city,country
0,2,Andrew,Fuller,Dr.,1952-02-19,1992-08-14,Tacoma,USA
1,5,Steven,Buchanan,Mr.,1955-03-04,1993-10-17,London,UK


In [13]:
res.repartition(1).write.csv('1a')

# Ejercicio 1b
1b. ¿Quién es el segundo “mejor” empleado que más órdenes ha generado? (nombre, apellido, título, cuándo entró a la compañía, número de órdenes generadas, número de órdenes generadas por el mejor empleado (número 1))

In [14]:
orders.createOrReplaceTempView("orders") 

In [15]:
res = spark.sql("""SELECT lastname, firstname, title, hiredate, count(*) 
as total_ordenes FROM orders JOIN employees 
ON orders.employeeid = employees.employeeid 
GROUP BY lastname, firstname, title, hiredate 
ORDER BY total_ordenes DESC LIMIT 2
""")
res.toPandas().loc[1]

lastname                    Leverling
firstname                       Janet
title            Sales Representative
hiredate                   1992-04-01
total_ordenes                     127
Name: 1, dtype: object

In [16]:
res.repartition(1).write.csv('1b')

# Ejercicio 1c
1c. ¿Cuál es el delta de tiempo más grande entre una orden y otra?

In [17]:
res = spark.sql("""
SELECT orderid, orderdate, lag(orderdate) OVER(ORDER BY orderid), 
datediff(orderdate, lag(orderdate) OVER(ORDER BY orderid)) delta
FROM orders 
ORDER BY delta DESC LIMIT 5
""")
res.toPandas()

,orderid,orderdate,"lag(orderdate, 1, NULL) OVER (ORDER BY orderid ASC NULLS FIRST ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING)",delta
0,10262,1996-07-22,1996-07-19,3
1,10284,1996-08-19,1996-08-16,3
2,10267,1996-07-29,1996-07-26,3
3,10256,1996-07-15,1996-07-12,3
4,10273,1996-08-05,1996-08-02,3


In [18]:
res.repartition(1).write.csv('1c')